In [ ]:
import decimal
import sys

import pandas

sys.path.append('..')

import src.constants
import src.hashstack
import src.nostra
import src.zklend
import update_data

In [ ]:
zklend_events = update_data.get_events(0)
hashstack_events = src.hashstack.get_hashstack_events()
nostra_events = src.nostra.get_nostra_events()

# Iterate over ordered events to obtain the final state of each user.
zklend_state = src.zklend.State()
for _, zklend_event in zklend_events.iterrows():
    zklend_state.process_event(event=zklend_event)
hashstack_state = src.hashstack.State()
for _, hashstack_event in hashstack_events.iterrows():
    hashstack_state.process_event(event=hashstack_event)
nostra_state = src.nostra.State()
for _, nostra_event in nostra_events.iterrows():
    nostra_state.process_event(event=nostra_event)

In [ ]:
prices = src.swap_liquidity.Prices()

swap_amms = await src.swap_liquidity.SwapAmm().init()

pairs = [
    ("ETH", "USDC"),
    ("ETH", "USDT"),
    ("ETH", "DAI"),
    ("wBTC", "USDC"),
    ("wBTC", "USDT"),
    ("wBTC", "DAI"),
]

In [ ]:
zklend_loan_stats = pandas.DataFrame()
zklend_loan_stats["User"] = [
    user
    for user in zklend_state.user_states.keys()
]
zklend_loan_stats["Protocol"] = "zkLend"
zklend_loan_stats["Borrowing in USD"] = zklend_loan_stats.apply(
    lambda x: src.zklend.compute_borrowings_usd(
        user_state=zklend_state.user_states[x["User"]],
        prices=prices.prices,
    ),
    axis=1,
)
zklend_loan_stats[
    "Risk adjusted collateral in USD"
] = zklend_loan_stats.apply(
    lambda x: src.zklend.compute_risk_adjusted_collateral_usd(
        user_state=zklend_state.user_states[x["User"]],
        prices=prices.prices,
    ),
    axis=1,
)
zklend_loan_stats["Health factor"] = zklend_loan_stats.apply(
    lambda x: src.zklend.compute_health_factor(
        risk_adjusted_collateral_usd=x["Risk adjusted collateral in USD"],
        borrowings_usd=x["Borrowing in USD"],
    ),
    axis=1,
)

In [ ]:
hashstack_loan_stats = pandas.DataFrame()
hashstack_loan_stats["User"] = [
    user
    for user, user_state in hashstack_state.user_states.items()
    for _ in user_state.loans.keys()
]
hashstack_loan_stats["Protocol"] = "Hashstack"
hashstack_loan_stats["Loan ID"] = [
    loan_id
    for user_state in hashstack_state.user_states.values()
    for loan_id in user_state.loans.keys()
]
hashstack_loan_stats["Borrowing in USD"] = hashstack_loan_stats.apply(
    lambda x: src.hashstack.compute_borrowings_amount_usd(
        borrowings=hashstack_state.user_states[x["User"]]
        .loans[x["Loan ID"]]
        .borrowings,
        prices=prices.prices,
    ),
    axis=1,
)
hashstack_loan_stats = hashstack_loan_stats[hashstack_loan_stats['Borrowing in USD'] > decimal.Decimal("0")]
hashstack_loan_stats[
    "Risk adjusted collateral in USD"
] = hashstack_loan_stats.apply(
    lambda x: src.hashstack.compute_collateral_current_amount_usd(
        collateral=hashstack_state.user_states[x["User"]]
        .loans[x["Loan ID"]]
        .collateral,
        prices=prices.prices,
    ),
    axis=1,
)
hashstack_loan_stats["Health factor"] = hashstack_loan_stats.apply(
    lambda x: src.hashstack.compute_health_factor(
        borrowings=hashstack_state.user_states[x["User"]]
        .loans[x["Loan ID"]]
        .borrowings,
        collateral=hashstack_state.user_states[x["User"]]
        .loans[x["Loan ID"]]
        .collateral,
        prices=prices.prices,
        user=x["User"],
    ),
    axis=1,
)
hashstack_loan_stats.drop(columns=["Loan ID"], inplace=True)

In [ ]:
nostra_loan_stats = pandas.DataFrame()
nostra_loan_stats["User"] = [
    user
    for user in nostra_state.user_states.keys()
]
nostra_loan_stats["Protocol"] = "Nostra"
nostra_loan_stats["Borrowing in USD"] = nostra_loan_stats.apply(
    lambda x: src.nostra.compute_borrowings_amount_usd(
        user_state=nostra_state.user_states[x["User"]],
        prices=prices.prices,
    ),
    axis=1,
)
nostra_loan_stats[
    "Risk adjusted collateral in USD"
] = nostra_loan_stats.apply(
    lambda x: src.nostra.compute_risk_adjusted_collateral_usd(
        user_state=nostra_state.user_states[x["User"]],
        prices=prices.prices,
    ),
    axis=1,
)
nostra_loan_stats["Health factor"] = nostra_loan_stats.apply(
    lambda x: src.nostra.compute_health_factor(
        risk_adjusted_collateral_usd=x["Risk adjusted collateral in USD"],
        risk_adjusted_debt_usd=x["Borrowing in USD"],
    ),
    axis=1,
)

In [ ]:
# TODO
comparison_stats = pandas.DataFrame(
    {
        'Protocal': [
            'zkLend',
            'Hashstack',
            'Nostra',
        ],
        'Number of users': [
            len(zklend_loan_stats),
            len(hashstack_loan_stats),
            len(nostra_loan_stats),
        ],
        'Total debt in USD': [
            round(zklend_loan_stats['Borrowing in USD'].sum(), 4),
            round(hashstack_loan_stats['Borrowing in USD'].sum(), 4),
            round(nostra_loan_stats['Borrowing in USD'].sum(), 4),
        ],
        'Total risk adjusted collateral in USD': [
            round(zklend_loan_stats['Risk adjusted collateral in USD'].sum(), 4),
            round(hashstack_loan_stats['Risk adjusted collateral in USD'].sum(), 4),
            round(nostra_loan_stats['Risk adjusted collateral in USD'].sum(), 4),
        ],
        'ETH debt': [
            round(sum(x.token_states['ETH'].borrowings for x in zklend_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['ETH'], 4),
            round(sum(loan.borrowings.amount for user_state in hashstack_state.user_states.values() for loan in user_state.loans.values() if loan.borrowings.market == 'ETH') / src.constants.TOKEN_DECIMAL_FACTORS['ETH'], 4),
            round(sum(x.token_states['ETH'].debt for x in nostra_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['ETH'], 4),
        ],
        'wBTC debt': [
            round(sum(x.token_states['wBTC'].borrowings for x in zklend_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['wBTC'], 4),
            round(sum(loan.borrowings.amount for user_state in hashstack_state.user_states.values() for loan in user_state.loans.values() if loan.borrowings.market == 'wBTC') / src.constants.TOKEN_DECIMAL_FACTORS['wBTC'], 4),
            round(sum(x.token_states['wBTC'].debt for x in nostra_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['wBTC'], 4),
        ],
        'USDC debt': [
            round(sum(x.token_states['USDC'].borrowings for x in zklend_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['USDC'], 4),
            round(sum(loan.borrowings.amount for user_state in hashstack_state.user_states.values() for loan in user_state.loans.values() if loan.borrowings.market == 'USDC') / src.constants.TOKEN_DECIMAL_FACTORS['USDC'], 4),
            round(sum(x.token_states['USDC'].debt for x in nostra_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['USDC'], 4),
        ],
        'DAI debt': [
            round(sum(x.token_states['DAI'].borrowings for x in zklend_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['DAI'], 4),
            round(sum(loan.borrowings.amount for user_state in hashstack_state.user_states.values() for loan in user_state.loans.values() if loan.borrowings.market == 'DAI') / src.constants.TOKEN_DECIMAL_FACTORS['DAI'], 4),
            round(sum(x.token_states['DAI'].debt for x in nostra_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['DAI'], 4),
        ],
        'USDT debt': [
            round(sum(x.token_states['USDT'].borrowings for x in zklend_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['USDT'], 4),
            round(sum(loan.borrowings.amount for user_state in hashstack_state.user_states.values() for loan in user_state.loans.values() if loan.borrowings.market == 'USDT') / src.constants.TOKEN_DECIMAL_FACTORS['USDT'], 4),
            round(sum(x.token_states['USDT'].debt for x in nostra_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['USDT'], 4),
        ],
        'ETH collateral': [
            round(sum(x.token_states['ETH'].deposit * x.token_states['ETH'].collateral_enabled for x in zklend_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['ETH'], 4),
            round(sum(loan.collateral.current_amount for user_state in hashstack_state.user_states.values() for loan in user_state.loans.values() if loan.collateral.market == 'ETH') / src.constants.TOKEN_DECIMAL_FACTORS['ETH'], 4),
            round(sum(x.token_states['ETH'].collateral + x.token_states['ETH'].interest_bearing_collateral for x in nostra_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['ETH'], 4),
        ],
        'wBTC collateral': [
            round(sum(x.token_states['wBTC'].deposit * x.token_states['wBTC'].collateral_enabled for x in zklend_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['wBTC'], 4),
            round(sum(loan.collateral.current_amount for user_state in hashstack_state.user_states.values() for loan in user_state.loans.values() if loan.collateral.market == 'wBTC') / src.constants.TOKEN_DECIMAL_FACTORS['wBTC'], 4),
            round(sum(x.token_states['wBTC'].collateral + x.token_states['wBTC'].interest_bearing_collateral for x in nostra_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['wBTC'], 4),
        ],
        'USDC collateral': [
            round(sum(x.token_states['USDC'].deposit * x.token_states['USDC'].collateral_enabled for x in zklend_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['USDC'], 4),
            round(sum(loan.collateral.current_amount for user_state in hashstack_state.user_states.values() for loan in user_state.loans.values() if loan.collateral.market == 'USDC') / src.constants.TOKEN_DECIMAL_FACTORS['USDC'], 4),
            round(sum(x.token_states['USDC'].collateral + x.token_states['USDC'].interest_bearing_collateral for x in nostra_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['USDC'], 4),
        ],
        'DAI collateral': [
            round(sum(x.token_states['DAI'].deposit * x.token_states['DAI'].collateral_enabled for x in zklend_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['DAI'], 4),
            round(sum(loan.collateral.current_amount for user_state in hashstack_state.user_states.values() for loan in user_state.loans.values() if loan.collateral.market == 'DAI') / src.constants.TOKEN_DECIMAL_FACTORS['DAI'], 4),
            round(sum(x.token_states['DAI'].collateral + x.token_states['DAI'].interest_bearing_collateral for x in nostra_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['DAI'], 4),
        ],
        'USDT collateral': [
            round(sum(x.token_states['USDT'].deposit * x.token_states['USDT'].collateral_enabled for x in zklend_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['USDT'], 4),
            round(sum(loan.collateral.current_amount for user_state in hashstack_state.user_states.values() for loan in user_state.loans.values() if loan.collateral.market == 'USDT') / src.constants.TOKEN_DECIMAL_FACTORS['USDT'], 4),
            round(sum(x.token_states['USDT'].collateral + x.token_states['USDT'].interest_bearing_collateral for x in nostra_state.user_states.values()) / src.constants.TOKEN_DECIMAL_FACTORS['USDT'], 4),
        ],
    },
)

In [ ]:
comparison_stats.to_csv("comparison_stats.csv", index=False)

In [ ]:
comparison_stats